## Import the required Libraries


In [1]:
import numpy as np
import keras
from keras.layers import Dense


Using TensorFlow backend.


## Initiate the Model

In [2]:
vgg_model=keras.applications.vgg16.VGG16()

Instructions for updating:
Colocations handled automatically by placer.
553467904/553467096 [==============================] - 428s 1us/step


## Check the model summary

In [3]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## Remove the last layer

In [4]:
last_layer = str(vgg_model.layers[-1])

classifier= keras.Sequential()
for layer in vgg_model.layers:
    if str(layer) != last_layer:
        classifier.add(layer)

## Recheck the summary & Last layer

In [5]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

## Freeze layers

In [6]:
for layer in classifier.layers:
    layer.trainable=False

# Add a new layer & Check summary & last layer

In [7]:
classifier.add(Dense(1,activation='sigmoid'))
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

## Complie the network

In [8]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

## Process the image and fit it

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

generate_train_data = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)





generate_test_data = ImageDataGenerator(rescale =1./255)

training_dataset = generate_train_data.flow_from_directory('dataset/training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_datasetset = generate_test_data.flow_from_directory('dataset/test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_dataset,
                         steps_per_epoch = 100,
                         epochs = 10,
                         validation_data = test_datasetset,
                         validation_steps = 30)

Found 200 images belonging to 2 classes.
Found 115 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
100/100 [==============================] - 1968s 20s/step - loss: 0.5427 - acc: 0.7369 - val_loss: 0.3629 - val_acc: 0.8516
Epoch 2/10
100/100 [==============================] - 1862s 19s/step - loss: 0.3922 - acc: 0.8478 - val_loss: 0.3751 - val_acc: 0.8329
Epoch 3/10
 56/100 [===============>..............] - ETA: 11:11 - loss: 0.3546 - acc: 0.8571

## Predict a new image

In [10]:
import numpy as np
from keras.preprocessing import image
new_image = image.load_img('../Data/Prediction/test_image_1.jpg', target_size = (224, 224))
new_image = image.img_to_array(new_image)
new_image = np.expand_dims(new_image, axis = 0)
result = classifier.predict(new_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'It is a Dog'
else:
    prediction = 'It is a Cat'

print(prediction)

It is a Cat
